In [1]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [5]:
# 1. 加载 labeldata.csv 和 标签的数据无打码.txt 文件
label_data = pd.read_csv('labeldata.csv')
no_mask_data = pd.read_csv('标签的数据无打码.txt', header=None, names=['text'], sep='\t')

In [6]:
# 2. 数据清洗：清除无效字符，统一文本格式
def clean_text(text):
    if not isinstance(text, str):
        return ''
    text = text.lower()  # 转小写
    text = re.sub(r'\d+', '', text)  # 去除数字
    text = re.sub(r'[^\w\s]', '', text)  # 去除标点符号
    return text

In [7]:
# 清洗文本数据
label_data['cleaned_text'] = label_data['信息的对华情感'].apply(clean_text)  # 清洗对华情感文本
no_mask_data['cleaned_text'] = no_mask_data['text'].apply(clean_text)  # 清洗预测文本

In [9]:
print(label_data.columns)

Index(['data_row', '信息的对华情感', '信息的私人情感（不指向CHN）', '作者是机构的可能性', 'train_predict',
       'cleaned_text'],
      dtype='object')


In [12]:
# 3. 合并数据：通过 data_row 进行合并，如果没有 data_row 则跳过此步骤
# 如果你有其他列来匹配文本和标签，修改合并逻辑
label_data = label_data.reset_index()  # 将原来的索引转化为列
merged_data = pd.merge(label_data[['index', '信息的对华情感', 'cleaned_text']], no_mask_data, left_on='index', right_index=True, how='left')

In [14]:
print(merged_data.columns)

Index(['index', '信息的对华情感', 'cleaned_text_x', 'text', 'cleaned_text_y'], dtype='object')


In [15]:
# 4. 特征提取：使用 TfidfVectorizer 将文本转化为特征向量
vectorizer = TfidfVectorizer(max_features=5000)  # 提取最多5000个特征
X = vectorizer.fit_transform(merged_data['cleaned_text_x'])

In [16]:
# 5. 准备标签数据
y = merged_data['信息的对华情感']  # 目标标签：对华情感

In [17]:
# 6. 切分训练集和预测集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [18]:
# 7. 训练情感预测模型：这里使用逻辑回归作为基线模型
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [19]:
# 8. 预测并评估模型
y_pred = model.predict(X_test)

In [20]:
# 9. 输出评估报告
print("Classification Report:\n")
print(classification_report(y_test, y_pred))

Classification Report:

              precision    recall  f1-score   support

           无       1.00      1.00      1.00        94
          消极       1.00      1.00      1.00        89
          积极       1.00      1.00      1.00        90

    accuracy                           1.00       273
   macro avg       1.00      1.00      1.00       273
weighted avg       1.00      1.00      1.00       273



In [21]:
# 10. 对于无打码的文本进行情感预测（预测集）
predict_X = vectorizer.transform(no_mask_data['cleaned_text'])
predict_y = model.predict(predict_X)

In [22]:
# 11. 将预测结果输出
no_mask_data['predicted_sentiment'] = predict_y
print("预测结果：")
print(no_mask_data[['text', 'predicted_sentiment']])

预测结果：
                                                   text predicted_sentiment
0          Area,KeyWords,Name,Verified,Language,Content                   无
1      军事,china cyber attack,jono,t,en,Breaking: For...                   无
2      军事,china cyber attack,Blutkanzlerin AngelaMer...                   无
3      军事,china cyber attack,kaneichi ,f,en,Chances ...                   无
4      军事,china cyber attack,Vishvaguru Citizen(Jeet...                   无
...                                                 ...                 ...
1360   经济,express delivery china,Algorithms R Us,f,e...                   无
1361   经济,express delivery china,Chin Ann,f,en,Wtf S...                   无
1362   经济,BeltandRoad,Abhijeet Rana,f,en,China belie...                   无
1363   经济,BeltandRoad,J∆Z ,t,en,What made me awe was...                   无
1364   经济,BeltandRoad,Sammy Thornton,f,en,Building a...                   无

[1365 rows x 2 columns]


In [23]:
# 12. 如果需要保存预测结果，可以使用下面的代码：
no_mask_data[['text', 'predicted_sentiment']].to_csv('predicted_results.csv', index=False)